<a href="https://colab.research.google.com/github/jbigham878/finalproject/blob/main/Final_Project_Steam_Game_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi & Welcome to my final project! Please see the below resources for some background on everything going on:

Project Proposal: https://docs.google.com/document/d/1ylmeU6Ckum71-FHI8Zi4qI6fSOrVFhjsrgGUMOjS-mM/edit?usp=sharing

Project Presentation (WIP): https://docs.google.com/presentation/d/1muA6QUlux4U_12DxF1i1UCzsPhvUcRoHYhPkoiaiqWs/edit?usp=sharing\

---

If you have any issues running the cells or questions about the below work, don't hesitate to reach out at jbigham878@gmail.com!

## **Imports & Definitions**

In [ ]:
! pip install xmltodict
! pip install kaggle
import pandas as pd
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
from itertools import chain

import json
import requests
import xmltodict

In [ ]:
!echo '{"username":"soulway","key":"7611763b64d674403c2d403a3126ab1f"}' > /content/kaggle.json

In [ ]:
! kaggle datasets download antonkozyriev/game-recommendations-on-steam
! mkdir game-recommendations-on-steam
! unzip game-recommendations-on-steam.zip -d game-recommendations-on-steam

100% 617M/618M [00:05<00:00, 132MB/s]
100% 618M/618M [00:05<00:00, 115MB/s]
Archive:  game-recommendations-on-steam.zip
  inflating: game-recommendations-on-steam/games.csv  
  inflating: game-recommendations-on-steam/games_metadata.json  
  inflating: game-recommendations-on-steam/recommendations.csv  
  inflating: game-recommendations-on-steam/users.csv  


## **Importing and Cleaning Data**

In [ ]:
# import data to dataframes

games = pd.read_csv('/content/game-recommendations-on-steam/games.csv')
#recommendations = pd.read_csv('/content/game-recommendations-on-steam/recommendations.csv')
#users = pd.read_csv('/content/game-recommendations-on-steam/users.csv')
games_metadata = pd.read_json('/content/game-recommendations-on-steam/games_metadata.json',lines=True)

In [ ]:
# remove rows where there are no tags for the game
games_metadata_cleaned = games_metadata[games_metadata['tags'].apply(lambda x: len(x) > 0)]

In [ ]:
#drop columns we don't need
games.drop(['win','mac','linux','price_final','price_original','discount','steam_deck'],axis=1, inplace=True)
#recommendations.drop(['helpful','funny'],axis=1, inplace=True)

In [ ]:
## REMOVE TMS ##


# check for nulls and remove them

for df in [games,games_metadata_cleaned]:

  null_check = df.isnull()

  for column in null_check.columns:
      print(null_check[column].value_counts())

# no null values

False    50796
Name: app_id, dtype: int64
False    50796
Name: title, dtype: int64
False    50796
Name: date_release, dtype: int64
False    50796
Name: rating, dtype: int64
False    50796
Name: positive_ratio, dtype: int64
False    50796
Name: user_reviews, dtype: int64
False    49625
Name: app_id, dtype: int64
False    49625
Name: description, dtype: int64
False    49625
Name: tags, dtype: int64


In [ ]:
#merge games and game_data to have one df with all the information about a game
game_data = pd.merge(games,games_metadata_cleaned, on =['app_id'])

# to minimize the dataset into the games we want, only include games with a 80+ rating and over 50 reviews
game_data_filtered1 = game_data.loc[(game_data['positive_ratio'] > 60) & (game_data['user_reviews'] > 100)]

#reset the index to 0 - 10383 after filtering down the dataframe
game_data_filtered = game_data_filtered1.reset_index(drop=True)

In [ ]:
# lookup in game_data_filtered
game_data_filtered.loc[game_data_filtered['title'] == 'Exit the Gungeon']


,app_id,title,date_release,rating,positive_ratio,user_reviews,description,tags
14596,1209490,Exit the Gungeon,2020-03-17,Mostly Positive,77,2722,"Exit the Gungeon is a small, spin-off 'dungeon...","[Action, Indie, Adventure, Arcade, Pixel Graph..."


## **Request User's Top Played Games Using Steam API**

In [ ]:
# steam web api key: 4AFC30E431BC74475C2C66AEF989A803

# example urls
# Get Steam id http://api.steampowered.com/ISteamUser/ResolveVanityURL/v0001/?key=XXXXXXXXXXXXXXXXXXXXXXX&vanityurl=userVanityUrlName
# GetOwnedGames  http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=XXXXXXXXXXXXXXXXX&steamid=76561197960434622&format=json
# example username: afarnsworth, soulway

In [ ]:
# Create a function to retrieve the user's game library using their steam_id

def getUserData():
  # Take input from user to get their username, which we will use to acquire their unique steam id
  steam_username = input('What is your steam username? ')

  username_r = requests.get('http://steamcommunity.com/id/{0}/games?tab=all&xml=1'.format(steam_username))

  dict_data = xmltodict.parse(username_r.content)

  # define url and key. extract steam_id from the json response
  steam_api_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
  steam_api_key = '4AFC30E431BC74475C2C66AEF989A803'
  steam_id = int(dict_data['gamesList']['steamID64'])

  parameters = {
      'key': steam_api_key,
      'steamid': steam_id,
      'format': 'json'
  }

  response = requests.get(steam_api_url,params=parameters)

  user_data_json = response.json()

  user_data_df = pd.DataFrame(user_data_json['response']['games'])

  # Sort the user's game library by the most played games

  user_data_sorted = user_data_df.sort_values(by='playtime_forever',ascending=False)

  user_data_sorted

  # create list of top 5 games by appid
  top_appids = user_data_sorted['appid'].value_counts().head(5).index.tolist()

  return top_appids

In [ ]:
for top_game in [730, 230410, 444090]:
  try:
    game_index = game_data_filtered.index[game_data_filtered['app_id'] == top_game][0]
    print(f"Game added to top game list")
    game_name = game_data_filtered.iloc[game_index]['title']
    print(game_name)
  except:
    print(f"One of your top 3 games not found. Will compute based on next most played games.")


#print("Your 3 most played games are: ", recommended_games)

One of your top 3 games not found. Will compute based on next most played games.
One of your top 3 games not found. Will compute based on next most played games.
One of your top 3 games not found. Will compute based on next most played games.


## **Build Recommender System using tf-idf vectorizer**

In [ ]:
# Content-Based Filtering using tfidf

tf = TfidfVectorizer(max_features=1500, analyzer='word', ngram_range=(1, 3), max_df = 0.8, min_df=5, stop_words='english')
tfidf_matrix = tf.fit_transform([str(i) for i in game_data_filtered['tags']])

In [ ]:
# from the tfidf matrix, find the games with similar tags using cosine similarity

cosine_similarities_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
def getSimilarGames_tfidf(top_5_games):
  # state the user's top 3 most played games

  most_played = []

  for top_game in top_5_games:
    try:
      game_index = game_data_filtered.index[game_data_filtered['app_id'] == top_game][0]
      game_name = game_data_filtered.iloc[game_index]['title']
      most_played.append(game_name)
    except IndexError:
          print(f"Game with app_id {top_game} not found in the database. Moving on to the next game.")\

  print('\nYour most played games are: ', most_played)
  # show the title and tags of the user's most played games
  most_played_df = game_data_filtered.loc[game_data_filtered['title'].isin(most_played)][['title', 'tags']]
  display(most_played_df)

  print('\nHere are some recommended games for you based on your playtime:')

  #create a list of top 30 games that we will append to each time we find similar games to a top played game
  top_thirty_games = []

  #check if top 3 games are in the database

  for game in most_played:

    # take in the user's top 3 played games, and one by one find similar games

    # retrive the index of the game from the dataframe based on app_id
    game_index = game_data_filtered.index[game_data_filtered['title'] == game][0]

    # get a list of how similar that game is to all other games
    similarity_scores = list(enumerate(cosine_similarities_tfidf[game_index]))

    # find the 10 most similar games
    similarity_scores.sort(key=lambda x: x[1], reverse=True)

    # take items 1-11 in the list and append them to the top 30 games
    top_thirty_games.append(similarity_scores[1:11])

  # flatten out the list of tuples into one big list
  top_thirty_games_flat = list(chain.from_iterable(top_thirty_games))

  #remove duplicates from top thirty games, then sort by the similarity rating
  top_thirty_games_no_duplicates = []
  unique_app_id_values=[]

  for i in top_thirty_games_flat:
    app_id_values = i[0]
    if app_id_values not in unique_app_id_values:
      unique_app_id_values.append(app_id_values)
      top_thirty_games_no_duplicates.append(i)

  top_thirty_games_no_duplicates.sort(key=lambda x: x[1], reverse=True)
  # print(top_thirty_games_no_duplicates)

  #take the top 10 most similar games
  top_10_recommended = top_thirty_games_no_duplicates[0:10]

  # find those top 10 games in the main game_data_filtered dataframe
  top_ten_games_index = [i[0] for i in top_10_recommended]
  top_10_recommended = game_data_filtered.iloc[top_ten_games_index]['title']

  # return dataframe so we can see the tags of the recommended games
  result = game_data_filtered[game_data_filtered.index.isin(top_ten_games_index)][['title', 'tags']]

  # prepare for output
  recommended_games = top_10_recommended.to_list()

  display(result)

  return


## **Build Recommender System using spaCy (NLP)**

In [ ]:
# Content-Based Filtering using spacy

import spacy.cli
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
# function to join the tag_lists into one big string
def join_tags(tag_list):
  tag_list_joined = ' '.join(tag_list)
  return tag_list_joined

# Function to calculate SpaCy similarity between two tag lists
def calculate_similarity(tag_list):
    tag_list_tokens = nlp(tag_list)
    return game_tags_tokens.similarity(tag_list_tokens)

In [ ]:
#add column to game_data_filtered that is the spacy vectors for the tags
game_data_filtered['tags_joined'] = game_data_filtered['tags'].apply(join_tags)

In [ ]:
Projedef getSimilarGames_spaCy(top_5_games):
  # state the user's top 3 most played games

  most_played = []

  for top_game in top_5_games:
    try:
      game_index = game_data_filtered.index[game_data_filtered['app_id'] == top_game][0]
      game_name = game_data_filtered.iloc[game_index]['title']
      most_played.append(game_name)
    except IndexError:
          print(f"Game with app_id {top_game} not found in the database. Moving on to the next game.")\

  print('\nYour most played games are: ', most_played)
  # show the title and tags of the user's most played games
  most_played_df = game_data_filtered.loc[game_data_filtered['title'].isin(most_played)][['title', 'tags']]
  #display(most_played_df)

  print('\nHere are some recommended games for you based on your playtime:')

  #create a list of top 30 games that we will append to each time we find similar games to a top played game
  top_thirty_games_dict = {}

  for game in most_played:
    print('starting iteration for: ', game)
    # reset tags_similarity column
    if 'tags_similarity' in game_data_filtered.columns:
      game_data_filtered.pop('tags_similarity')
      print('tags_similarity reset')


    # retrieve the game_tags that we are going to compare against all the other ones
    game_tags = game_data_filtered.loc[game_data_filtered['title'] == game]['tags_joined'].values[0]
    #print(game_tags)

    # Tokenize the game_tags
    game_tags_tokens = nlp(game_tags)

    def calculate_similarity(tag_list):
      tag_list_tokens = nlp(tag_list)
      return game_tags_tokens.similarity(tag_list_tokens)

    # Calculate similarity scores from game_tags against all the other games. Add them to the df as the 'tags_similarity' column
    game_data_filtered['tags_similarity'] = game_data_filtered['tags_joined'].apply(calculate_similarity)

    # Sort the DataFrame based on tags_similarity and get the top 10 most similar rows
    top_10_similar = game_data_filtered.sort_values(by='tags_similarity', ascending=False).head(11) #display(top_10_similar)

    for index, row in top_10_similar.iterrows():
      top_thirty_games_dict[index] = row['tags_similarity']

  # flatten out the list of tuples into one big list
  #top_thirty_games_flat = list(chain.from_iterable(top_thirty_games)) #print('top_thirty_games_flat', top_thirty_games_flat)
  top_thirty_games_ordered = dict(sorted(top_thirty_games_dict.items(), key=lambda item: item[1], reverse=True)) #display(top_thirty_games_ordered)

  # take the top 10 most similar games fropm the top 30 games
  top_ten_games_index = list(top_thirty_games_ordered.keys())[:13] #print('top_ten_games_index',top_ten_games_index)

  # return dataframe so we can see the tags of the recommended games
  result_df_spacy = game_data_filtered[game_data_filtered.index.isin(top_ten_games_index)][['app_id','title', 'tags','tags_similarity']]

  # remove any games that are in the most_played list because we don't want to recommend the same games (they will have a similarity score of 1.0!)
  result_spacy = result_df_spacy[~result_df_spacy['title'].isin(most_played)]

  display(result_spacy)

  return


## **Test Cases**

In [ ]:
# tf-idf test case 1: Zombie Games

# 304930 Unturned
# 226560 Escape Dead Island
# 1250 Killing Floor

pd.set_option('display.max_colwidth', 500)

test_case_1_games = [304930,226560,1250]

getSimilarGames_tfidf(test_case_1_games)



Your most played games are:  ['Unturned', 'Escape Dead Island', 'Killing Floor']


,title,tags
2,Escape Dead Island,"[Zombies, Adventure, Survival, Action, Third Person, Open World, Gore, Horror, Singleplayer, Multiplayer, Cartoony, Hack and Slash, Stealth, Co-op, Atmospheric, Survival Horror, Third-Person Shooter, Anime]"
3391,Killing Floor,"[FPS, Zombies, Co-op, Survival, Horror, Action, Multiplayer, Online Co-Op, Gore, Shooter, Team-Based, First-Person, Survival Horror, Moddable, Great Soundtrack, Class-Based, Difficult, Singleplayer, Comedy, Adventure]"
13993,Unturned,"[Free to Play, Survival, Zombies, Multiplayer, Open World, Co-op, Crafting, Sandbox, Adventure, Shooter, First-Person, Post-apocalyptic, Looter Shooter, Action, FPS, Singleplayer, Massively Multiplayer, Atmospheric, Indie, Casual]"



Here are some recommended games for you based on your playtime:


,title,tags
481,Afterinfection,"[Early Access, Zombies, Survival, MMORPG, Open World Survival Craft, Looter Shooter, Massively Multiplayer, Open World, Third-Person Shooter, FPS, Outbreak Sim, Post-apocalyptic, Hidden Object, Hunting, PvP, Shooter, Crafting, RPG, Third Person, First-Person]"
2524,Nemesis: Distress,"[Survival Horror, FPS, Shooter, First-Person, Strategy, 3D, Action, Horror, Team-Based, Realistic, Space, Sci-fi, Survival, Beautiful, PvP, Co-op, Multiplayer, Online Co-Op, Early Access, Gore]"
4453,Unturned - Permanent Gold Upgrade,"[Free to Play, Survival, Zombies, Multiplayer, Open World, Adventure, Action, Crafting, Sandbox, Shooter, Co-op, FPS, Singleplayer, Post-apocalyptic, Indie, Casual, Funny, Massively Multiplayer, RPG, Horror]"
5241,The Goatman,"[Action, Adventure, Horror, Online Co-Op, Gore, Hunting, Strategy, Simulation, Multiplayer, Survival, Co-op, Survival Horror, Atmospheric, Team-Based, Psychological Horror, Third Person, Casual, First-Person, Open World]"
10290,Zombie Panic! Source,"[Free to Play, Zombies, Multiplayer, Action, Survival, Horror, FPS, Mod, Indie, Co-op, First-Person, Online Co-Op, Shooter, Team-Based, Survival Horror, Adventure, Casual, Gore, Atmospheric, Dark]"
11447,Pound of Ground,"[Action, Zombies, Third-Person Shooter, Singleplayer, Third Person, Open World]"
12734,Alien Swarm,"[Free to Play, Co-op, Action, Multiplayer, Online Co-Op, Aliens, Shooter, Sci-fi, Top-Down, Top-Down Shooter, Survival, PvE, Class-Based, Third Person, Team-Based, Third-Person Shooter, Moddable, Singleplayer, Strategy, Adventure]"
14037,Tom Clancy’s The Division™,"[Open World, Looter Shooter, Multiplayer, Third-Person Shooter, Action, Co-op, Shooter, RPG, Online Co-Op, Post-apocalyptic, Survival, Massively Multiplayer, Tactical, Third Person, Singleplayer, MMORPG, Adventure, Atmospheric, FPS, Stealth]"
14276,Will To Live Online,"[Survival, Multiplayer, Open World, Free to Play, Post-apocalyptic, Open World Survival Craft, Massively Multiplayer, FPS, MMORPG, Shooter, Action, Crafting, First-Person, RPG, Early Access, Exploration, Atmospheric, Gore, Violent, Indie]"
14302,After the Fall®,"[VR, Action, Co-op, Adventure, Zombies, Gore, FPS, Action-Adventure, Shooter, Looter Shooter, First-Person, Shoot 'Em Up, Realistic, Post-apocalyptic, 1980s, Crafting, Survival Horror, Dystopian, Multiplayer, Violent]"


In [ ]:
# Spacy test case 1: Zombie Games

test_case_1_games = [304930,226560,1250]

getSimilarGames_spaCy(test_case_1_games)


Your most played games are:  ['Unturned', 'Escape Dead Island', 'Killing Floor']

Here are some recommended games for you based on your playtime:
starting iteration for:  Unturned
starting iteration for:  Escape Dead Island
tags_similarity reset
starting iteration for:  Killing Floor
tags_similarity reset


,app_id,title,tags,tags_similarity
3358,608800,Guns of Icarus Alliance,"[Indie, Action, Adventure, Simulation, Steampunk, Multiplayer, Team-Based, Co-op, First-Person, Competitive, Great Soundtrack, Shooter, Online Co-Op, Family Friendly, FPS, Flight, PvE]",0.983642
4453,306460,Unturned - Permanent Gold Upgrade,"[Free to Play, Survival, Zombies, Multiplayer, Open World, Adventure, Action, Crafting, Sandbox, Shooter, Co-op, FPS, Singleplayer, Post-apocalyptic, Indie, Casual, Funny, Massively Multiplayer, RPG, Horror]",0.894560
4555,306690,PAYDAY 2: The Big Bank Heist,"[Action, FPS, Co-op, Heist, Stealth, Online Co-Op, RPG, Shooter, Multiplayer, Crime, Great Soundtrack, First-Person, Strategy, Team-Based]",0.980370
6296,257420,Serious Sam 4,"[Action, Gore, FPS, Co-op, Shooter, Violent, Multiplayer, Adventure, Comedy, Singleplayer, First-Person, Blood, Fast-Paced, Indie, Online Co-Op, Difficult, Arena Shooter, Horror, Demons, Third-Person Shooter]",0.981104
6555,1577320,Somewhere in the Shadow,"[Horror, Online Co-Op, Multiplayer, Co-op, Adventure, Survival Horror, Artificial Intelligence, Puzzle, Exploration, Violent, Action-Adventure, First-Person, Indie, Massively Multiplayer, Mystery Dungeon, Mystery, Team-Based, Singleplayer]",0.985925
7598,1049800,BLOCKADE,"[Free to Play, Action, FPS, Shooter, Multiplayer, Pixel Graphics, Massively Multiplayer, Indie, First-Person, Co-op, Zombies, Building, Early Access, Psychological Horror, Team-Based, Survival, Casual, Horror, Gore, Open World]",0.932111
7764,24240,PAYDAY™ The Heist,"[FPS, Action, Co-op, Heist, Multiplayer, Crime, Online Co-Op, Shooter, First-Person, Great Soundtrack, Team-Based, Stealth, Difficult, Tactical, Strategy, Singleplayer, Simulation, RPG, Indie, Adventure]",0.980260
8906,1850740,Ghost Watchers,"[Horror, Co-op, Survival Horror, Adventure, Demons, Puzzle, Multiplayer, First-Person, Action-Adventure, Cinematic, Dark, Realistic, Detective, PvE, Online Co-Op, Team-Based, Nonlinear, Singleplayer, Early Access, VR]",0.982011
11219,224260,No More Room in Hell,"[Free to Play, Zombies, Multiplayer, Survival, Horror, Co-op, Online Co-Op, Action, FPS, First-Person, Survival Horror, Shooter, Post-apocalyptic, Open World, Realistic, Indie, Atmospheric, Mod, Adventure, Massively Multiplayer]",0.961028
11221,293540,Guns and Robots,"[Free to Play, Action, Robots, Multiplayer, Shooter, Third-Person Shooter, Massively Multiplayer, Sandbox, Indie, FPS, Open World, Co-op, RPG, Building, Survival, Strategy, Adventure, First-Person, Singleplayer, Casual]",0.914913


In [ ]:
# tf-idf test case 2: indie games

test_case_2_games = [113200,1092840,1209490]

getSimilarGames_tfidf(test_case_2_games)


Your most played games are:  ['The Binding of Isaac', 'Celeste Soundtrack', 'Exit the Gungeon']


,title,tags
7899,Celeste Soundtrack,"[Action, Adventure, Indie, Soundtrack, Great Soundtrack]"
13875,The Binding of Isaac,"[Action Roguelike, Roguelike, Indie, Replay Value, Difficult, Procedural Generation, Dark, Action, Top-Down, Great Soundtrack, Addictive, Singleplayer, Adventure, Roguelite, 2D, Dungeon Crawler, Survival, RPG, Perma Death, Bullet Hell]"
14596,Exit the Gungeon,"[Action, Indie, Adventure, Arcade, Pixel Graphics, Bullet Hell, 2D, Shooter, Difficult, Singleplayer, 2D Platformer, Dungeon Crawler, Funny, Great Soundtrack, Retro, Atmospheric, Multiplayer, Co-op, Metroidvania, Roguelike]"



Here are some recommended games for you based on your playtime:


,title,tags
545,Tower of Guns,"[FPS, Action Roguelike, Roguelike, Indie, Action, Roguelite, Bullet Hell, Shooter, Singleplayer, First-Person, Perma Death, Procedural Generation, Fast-Paced, Replay Value, Difficult, Steampunk, Great Soundtrack]"
1126,DOOM Soundtrack,"[Action, Soundtrack, Great Soundtrack]"
4118,Straimium Immortaly,"[Action Roguelike, Roguelite, Indie, Roguelike, Action, Difficult, Bullet Hell, Dungeon Crawler, Shoot 'Em Up, Replay Value, Great Soundtrack, Pixel Graphics, Procedural Generation, Perma Death, Shooter, Atmospheric, Psychedelic, 2D, Ninja]"
4981,GRIS Soundtrack,"[Adventure, Indie, Soundtrack]"
5192,Ori and the Will of the Wisps Soundtrack,"[Soundtrack, Great Soundtrack]"
7281,Hollow Knight - Official Soundtrack,"[Action, Adventure, Indie, Soundtrack, Metroidvania, Great Soundtrack, 2D]"
7899,Celeste Soundtrack,"[Action, Adventure, Indie, Soundtrack, Great Soundtrack]"
9448,A Hat in Time - B-Side Soundtrack,"[Adventure, Indie, Soundtrack]"
11677,Aragami - Assassin Masks Set,"[Action, Adventure, Indie, Stealth]"
12195,Boo Bunny Plague,"[Action, Adventure, Indie, Comedy, Sci-fi, Music, Great Soundtrack, Funny]"


In [ ]:
# spaCytest case 2: indie games

test_case_2_games = [113200,1092840,1209490]
getSimilarGames_spaCy(test_case_2_games)


Your most played games are:  ['The Binding of Isaac', 'Celeste Soundtrack', 'Exit the Gungeon']

Here are some recommended games for you based on your playtime:
starting iteration for:  The Binding of Isaac
tags_similarity reset
starting iteration for:  Celeste Soundtrack
tags_similarity reset
starting iteration for:  Exit the Gungeon
tags_similarity reset


,app_id,title,tags,tags_similarity
804,704710,Cuphead - Official Soundtrack,"[Indie, Action, Soundtrack, Great Soundtrack, Difficult]",0.701804
1758,916000,Hollow Knight - Gods & Nightmares,"[Indie, Action, Adventure, Soundtrack, Great Soundtrack, Metroidvania, Difficult]",0.742772
1948,685400,Skelly Selest,"[Action Roguelike, Roguelite, Indie, Roguelike, Action, Pixel Graphics, Adventure, Hack and Slash, Dungeon Crawler, Procedural Generation, Difficult, Singleplayer, Replay Value, Top-Down, Great Soundtrack, Perma Death, Atmospheric, GameMaker, Card Game, 2D]",0.889108
1978,431620,Rabi-Ribi - Original Soundtrack,"[Action, Indie, Adventure, Great Soundtrack, Cute, Soundtrack]",0.729642
2308,366420,UBERMOSH: Original Soundtrack,"[Action, Indie, Great Soundtrack]",0.676600
3511,322020,The Talos Principle - Soundtrack,"[Action, Adventure, Indie, Soundtrack, Great Soundtrack]",0.719484
4067,506870,Unexplored,"[Roguelite, Action Roguelike, RPG, Indie, Adventure, Action, Procedural Generation, Dungeon Crawler, Perma Death, Replay Value, Action RPG, Great Soundtrack, Top-Down, 2D, Roguelike, Singleplayer]",0.879272
4430,1040420,Dreamscaper,"[Action Roguelike, Action, Roguelite, Indie, RPG, Dungeon Crawler, Singleplayer, Replay Value, Adventure, Great Soundtrack, Female Protagonist, Action RPG, Difficult, Hack and Slash, Roguelike, Atmospheric, Story Rich, Fantasy, Surreal, Top-Down]",0.855500
7726,219152,Hotline Miami Soundtrack,"[Indie, Action, Great Soundtrack, Soundtrack]",0.692449
8863,113204,Binding of Isaac: Wrath of the Lamb,"[Indie, Action, Adventure, RPG, Roguelike, Difficult, Singleplayer, Roguelite, Replay Value, Procedural Generation, Great Soundtrack, 2D, Dungeon Crawler, Horror, Dark, Gore, Addictive, Top-Down, Bullet Hell]",0.885642


In [ ]:
# tf-idf test case 3: FPS adventure

# 70 Half Life
# 22300 Fallout 3
# 220240 Far Cry 3

test_case_3_games = [70,22300,220240]

getSimilarGames_tfidf(test_case_3_games)


Your most played games are:  ['Half-Life', 'Fallout 3', 'Far Cry 3']


,title,tags
13788,Half-Life,"[FPS, Sci-fi, Multiplayer, 1990's, Classic, Singleplayer, Action, Shooter, First-Person, Aliens, Silent Protagonist, Story Rich, Atmospheric, Moddable, Adventure, Retro, Action-Adventure, Gore, Difficult, PvP]"
13828,Fallout 3,"[Open World, Post-apocalyptic, RPG, Singleplayer, Exploration, First-Person, Sci-fi, Adventure, Moddable, Action, Atmospheric, FPS, Shooter, Character Customization, Third Person, Story Rich, Sandbox, Action RPG, Horror, Female Protagonist]"
13905,Far Cry 3,"[Open World, FPS, Action, Adventure, Multiplayer, Shooter, Stealth, Co-op, First-Person, Hunting, Singleplayer, Sandbox, Survival, Story Rich, Online Co-Op, Atmospheric, Nudity, Crafting, Colorful, RPG]"



Here are some recommended games for you based on your playtime:


,title,tags
781,Homefront®: The Revolution,"[Action, FPS, Open World, Multiplayer, Shooter, Co-op, First-Person, War, Singleplayer, Online Co-Op, Stealth, Adventure, Military, Survival, Atmospheric, Dystopian, Crafting, Futuristic, Story Rich, Violent]"
2811,Fallout 4 - Automatron,"[RPG, Robots, Post-apocalyptic, Open World, Action, Singleplayer, Adventure, FPS, Atmospheric, Sci-fi, Sandbox, Shooter, First-Person, Story Rich]"
3641,Deadly Broadcast,"[FPS, Action, Adventure, Multiplayer, Horror, Co-op, Survival Horror, Supernatural, 3D, Survival, Online Co-Op, Demons, Thriller, Psychological Horror, Surreal, Mystery, Zombies, First-Person, Dark, Action-Adventure]"
4111,Arma 3 Apex,"[Simulation, Action, Military, Multiplayer, Strategy, Open World, FPS, Sandbox, Tactical, Co-op, Realistic, Shooter, Singleplayer, Survival, First-Person, Online Co-Op, Third-Person Shooter, Atmospheric, Walking Simulator, RPG]"
4316,Fallout 4 Far Harbor,"[RPG, Open World, Post-apocalyptic, Singleplayer, Survival, FPS, Shooter, Atmospheric, Action, Adventure, Action RPG, Sci-fi, Exploration, Lovecraftian, Third Person, Crafting, First-Person, Great Soundtrack, Sandbox, Story Rich]"
7731,Saints Row: Gat out of Hell,"[Action, Open World, Comedy, Co-op, Third Person, Funny, Demons, Third-Person Shooter, Adventure, Multiplayer, Shooter, Singleplayer, Sandbox, Dark Humor, Atmospheric, Female Protagonist, Mature, Online Co-Op, Gore, Nudity]"
8393,PAYDAY 2: Gage Weapon Pack #01,"[Action, RPG, FPS, Co-op, Heist, Gun Customization, Multiplayer, Shooter, Online Co-Op, First-Person]"
10772,Alien Rage - Unlimited,"[Action, Shooter, FPS, First-Person, Sci-fi, Multiplayer, Aliens, Singleplayer, Difficult, Arcade]"
13983,Far Cry® 4,"[Open World, Action, FPS, Co-op, Adventure, Shooter, Multiplayer, First-Person, Singleplayer, Hunting, Stealth, Survival, Atmospheric, Online Co-Op, Story Rich, Exploration, Crafting, Sandbox, RPG, Action RPG]"
14209,BIOMUTANT,"[Open World, Action, RPG, Singleplayer, Adventure, Post-apocalyptic, Character Customization, Third Person, Hack and Slash, Fantasy, Atmospheric, Action RPG, Exploration, Martial Arts, Multiplayer, Cute, Story Rich, Sci-fi, Indie, Crafting]"


In [ ]:
# spaCy test case 3: FPS adventure

test_case_3_games = [70,22300,220240]

getSimilarGames_spaCy(test_case_3_games)


Your most played games are:  ['Half-Life', 'Fallout 3', 'Far Cry 3']

Here are some recommended games for you based on your playtime:
starting iteration for:  Half-Life
tags_similarity reset
starting iteration for:  Fallout 3
tags_similarity reset
starting iteration for:  Far Cry 3
tags_similarity reset


,app_id,title,tags,tags_similarity
759,939960,Far Cry® New Dawn,"[Open World, FPS, Action, Co-op, Post-apocalyptic, Multiplayer, Adventure, First-Person, Shooter, Online Co-Op, Singleplayer, Stealth, Story Rich, Survival, RPG, Female Protagonist, Sandbox, Action RPG, Casual, Psychological Horror]",0.986769
4111,395180,Arma 3 Apex,"[Simulation, Action, Military, Multiplayer, Strategy, Open World, FPS, Sandbox, Tactical, Co-op, Realistic, Shooter, Singleplayer, Survival, First-Person, Online Co-Op, Third-Person Shooter, Atmospheric, Walking Simulator, RPG]",0.980991
4316,435881,Fallout 4 Far Harbor,"[RPG, Open World, Post-apocalyptic, Singleplayer, Survival, FPS, Shooter, Atmospheric, Action, Adventure, Action RPG, Sci-fi, Exploration, Lovecraftian, Third Person, Crafting, First-Person, Great Soundtrack, Sandbox, Story Rich]",0.869676
4831,1206600,Almighty: Kill Your Gods,"[Early Access, Looter Shooter, Hack and Slash, Souls-like, Collectathon, Action RPG, RPG, Third Person, Open World, Action, Fantasy, Atmospheric, Crafting, Adventure, Co-op, Co-op Campaign, Multiplayer, Online Co-Op, Singleplayer, Indie]",0.984078
5127,541300,Survive the Nights,"[Early Access, Post-apocalyptic, RPG, Action, Realistic, Online Co-Op, Massively Multiplayer, Combat, Inventory Management, Co-op, Survival, Crafting, Zombies, Horror, Exploration, Open World, FPS, Shooter, Sandbox, PvE]",0.982931
6571,386880,Rake,"[Horror, Multiplayer, Survival Horror, Survival, First-Person, Hunting, Open World, Action, Indie, Adventure, Co-op, Atmospheric, Singleplayer, Simulation, FPS, Shooter, Gore, Supernatural, Online Co-Op, Funny]",0.986910
7398,810040,Swords 'n Magic and Stuff,"[Singleplayer, Co-op, Early Access, Medieval, Casual, Fantasy, Adventure, RPG, Exploration, Online Co-Op, Open World, Building, Multiplayer, Third Person, Loot, Cute, Story Rich, Sandbox, Action-Adventure, Magic]",0.982610
13983,298110,Far Cry® 4,"[Open World, Action, FPS, Co-op, Adventure, Shooter, Multiplayer, First-Person, Singleplayer, Hunting, Stealth, Survival, Atmospheric, Online Co-Op, Story Rich, Exploration, Crafting, Sandbox, RPG, Action RPG]",0.994412
14056,383150,Dead Island Definitive Edition,"[Zombies, Open World, Co-op, Multiplayer, Survival, Action, Gore, Horror, First-Person, FPS, Online Co-Op, Singleplayer, Crafting, RPG, Survival Horror, Adventure, Nudity, Violent, Controller, Female Protagonist]",0.985193
14725,1404210,Red Dead Online,"[Open World, Multiplayer, Action, Western, Adventure, Horses, Shooter, Realistic, Online Co-Op, Third-Person Shooter, PvP, Character Customization, Story Rich, Singleplayer, Co-op, MMORPG, Violent, First-Person, Controller, Gore]",0.982216


In [ ]:
# tf-idf test case 3: Mixed

test_case_4_games = [113200,759,220240]

getSimilarGames_tfidf(test_case_4_games)

Game with app_id 759 not found in the database. Moving on to the next game.

Your most played games are:  ['The Binding of Isaac', 'Far Cry 3']


,title,tags
13875,The Binding of Isaac,"[Action Roguelike, Roguelike, Indie, Replay Value, Difficult, Procedural Generation, Dark, Action, Top-Down, Great Soundtrack, Addictive, Singleplayer, Adventure, Roguelite, 2D, Dungeon Crawler, Survival, RPG, Perma Death, Bullet Hell]"
13905,Far Cry 3,"[Open World, FPS, Action, Adventure, Multiplayer, Shooter, Stealth, Co-op, First-Person, Hunting, Singleplayer, Sandbox, Survival, Story Rich, Online Co-Op, Atmospheric, Nudity, Crafting, Colorful, RPG]"



Here are some recommended games for you based on your playtime:


,title,tags
545,Tower of Guns,"[FPS, Action Roguelike, Roguelike, Indie, Action, Roguelite, Bullet Hell, Shooter, Singleplayer, First-Person, Perma Death, Procedural Generation, Fast-Paced, Replay Value, Difficult, Steampunk, Great Soundtrack]"
714,Dandy Ace,"[Action, Indie, Roguelite, Isometric, Roguelike, Difficult, Dungeon Crawler, Singleplayer, Magic, Perma Death, Procedural Generation, Replay Value, Atmospheric, Hand-drawn, 2D]"
1685,Heavy Bullets,"[Indie, Action, Action Roguelike, FPS, Roguelike, Shooter, Roguelite, Singleplayer, First-Person, Difficult, Retro, Dungeon Crawler, Perma Death, Horror, Psychedelic, Colorful, Procedural Generation, Adventure, Bullet Hell]"
2475,Spelunky,"[Platformer, Action Roguelike, Roguelike, Indie, Difficult, 2D, Local Co-Op, Replay Value, Procedural Generation, Addictive, Action, Perma Death, Singleplayer, Adventure, Co-op, Roguelite, Controller, Local Multiplayer, Great Soundtrack, Arcade]"
4067,Unexplored,"[Roguelite, Action Roguelike, RPG, Indie, Adventure, Action, Procedural Generation, Dungeon Crawler, Perma Death, Replay Value, Action RPG, Great Soundtrack, Top-Down, 2D, Roguelike, Singleplayer]"
4118,Straimium Immortaly,"[Action Roguelike, Roguelite, Indie, Roguelike, Action, Difficult, Bullet Hell, Dungeon Crawler, Shoot 'Em Up, Replay Value, Great Soundtrack, Pixel Graphics, Procedural Generation, Perma Death, Shooter, Atmospheric, Psychedelic, 2D, Ninja]"
7179,Delver,"[Dungeon Crawler, First-Person, RPG, Action Roguelike, Indie, Pixel Graphics, Roguelite, Adventure, Action, Roguelike, Perma Death, Singleplayer, Procedural Generation, Great Soundtrack, Retro, FPS, Early Access, Moddable, Fantasy, Atmospheric]"
8369,ScourgeBringer,"[Indie, Action, Adventure, Roguelite, Pixel Graphics, Action Roguelike, Roguelike, Roguevania, 2D, Platformer, Metroidvania, Singleplayer, Bullet Hell, Procedural Generation, Difficult, Replay Value, Great Soundtrack, Perma Death, Female Protagonist, Fast-Paced]"
11741,The Madness of Little Emma,"[Action Roguelike, Indie, Action, Platformer, Female Protagonist, Roguelike, Great Soundtrack, Replay Value, Difficult, 2D, Horror, RPG, Roguelite, Bullet Hell]"
12685,Enlightenment,"[Action, Indie, Action Roguelike, Roguelite, Top-Down, Survival, Atmospheric, RPG, Exploration, Dark, Difficult, Controller, Roguelike, Procedural Generation, Perma Death, Dungeon Crawler, Early Access, Replay Value, Adventure]"


In [ ]:
# spaCy test case 3: Mixed

test_case_4_games = [113200,759,220240]

getSimilarGames_spaCy(test_case_4_games)

Game with app_id 759 not found in the database. Moving on to the next game.

Your most played games are:  ['The Binding of Isaac', 'Far Cry 3']

Here are some recommended games for you based on your playtime:
starting iteration for:  The Binding of Isaac
tags_similarity reset
starting iteration for:  Far Cry 3
tags_similarity reset


,app_id,title,tags,tags_similarity
759,939960,Far Cry® New Dawn,"[Open World, FPS, Action, Co-op, Post-apocalyptic, Multiplayer, Adventure, First-Person, Shooter, Online Co-Op, Singleplayer, Stealth, Story Rich, Survival, RPG, Female Protagonist, Sandbox, Action RPG, Casual, Psychological Horror]",0.986769
4067,506870,Unexplored,"[Roguelite, Action Roguelike, RPG, Indie, Adventure, Action, Procedural Generation, Dungeon Crawler, Perma Death, Replay Value, Action RPG, Great Soundtrack, Top-Down, 2D, Roguelike, Singleplayer]",0.787430
4111,395180,Arma 3 Apex,"[Simulation, Action, Military, Multiplayer, Strategy, Open World, FPS, Sandbox, Tactical, Co-op, Realistic, Shooter, Singleplayer, Survival, First-Person, Online Co-Op, Third-Person Shooter, Atmospheric, Walking Simulator, RPG]",0.980991
4831,1206600,Almighty: Kill Your Gods,"[Early Access, Looter Shooter, Hack and Slash, Souls-like, Collectathon, Action RPG, RPG, Third Person, Open World, Action, Fantasy, Atmospheric, Crafting, Adventure, Co-op, Co-op Campaign, Multiplayer, Online Co-Op, Singleplayer, Indie]",0.984078
5127,541300,Survive the Nights,"[Early Access, Post-apocalyptic, RPG, Action, Realistic, Online Co-Op, Massively Multiplayer, Combat, Inventory Management, Co-op, Survival, Crafting, Zombies, Horror, Exploration, Open World, FPS, Shooter, Sandbox, PvE]",0.982931
6571,386880,Rake,"[Horror, Multiplayer, Survival Horror, Survival, First-Person, Hunting, Open World, Action, Indie, Adventure, Co-op, Atmospheric, Singleplayer, Simulation, FPS, Shooter, Gore, Supernatural, Online Co-Op, Funny]",0.986910
7398,810040,Swords 'n Magic and Stuff,"[Singleplayer, Co-op, Early Access, Medieval, Casual, Fantasy, Adventure, RPG, Exploration, Online Co-Op, Open World, Building, Multiplayer, Third Person, Loot, Cute, Story Rich, Sandbox, Action-Adventure, Magic]",0.982610
8863,113204,Binding of Isaac: Wrath of the Lamb,"[Indie, Action, Adventure, RPG, Roguelike, Difficult, Singleplayer, Roguelite, Replay Value, Procedural Generation, Great Soundtrack, 2D, Dungeon Crawler, Horror, Dark, Gore, Addictive, Top-Down, Bullet Hell]",0.762375
13983,298110,Far Cry® 4,"[Open World, Action, FPS, Co-op, Adventure, Shooter, Multiplayer, First-Person, Singleplayer, Hunting, Stealth, Survival, Atmospheric, Online Co-Op, Story Rich, Exploration, Crafting, Sandbox, RPG, Action RPG]",0.994412
14056,383150,Dead Island Definitive Edition,"[Zombies, Open World, Co-op, Multiplayer, Survival, Action, Gore, Horror, First-Person, FPS, Online Co-Op, Singleplayer, Crafting, RPG, Survival Horror, Adventure, Nudity, Violent, Controller, Female Protagonist]",0.985193


## **Analysis & Conclusion**

Both methods return accurate recommendations that share tags with all of the most played games. If I were to recommend one method over the other, the tf-idf vectorization implementation is much faster, returning recommendations in seconds rather than up to 4 minutes for the spaCy implementation.

It is difficult to verify which model is better due to the fact that there are no metrics or test data to compare against. I used my knowledge of the recommended games and also an eye test to see if they all share similar tags to verify that the recommended games in the output are appropriate for the input.


---


**Improvements to be made:**
- The dataset used had a weird gap: some of the most popular games did not have associated tags. It was a weird trend because you'd think the most popular and well-known games would have their tags associated - but tere was a data gap there. An improvement would be for a more full dataset (or if Steam could implement an API that would allow us to scrape the tags from their application!)

- spaCy implementation currently has a lot of runtime processes which makes it run for a long time before giving an output. I think if I made a matrix of all the tags comparison and front-loaded the processing, it would be a lot more efficient.